# Registros de empleados

Se creara una aplicacion grafica con Python que gestione una base de datos de "empleados", entendiendose por gestion la Conexión a la base de datos, Creación de la tabla de empleados,Inserción, Consulta, actualización y Eliminación de un empleado y su informacion dentro de la empresa.

In [1]:
### Importacion de librerias
import sqlite3
from sqlite3 import Error
from tkinter import *
from tkinter import messagebox

# Creación de la conexión a la base de datos

In [2]:
def crear_conexion():
    conn = None;
    try:
        conn = sqlite3.connect('empleados_db.sqlite')
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

# Creación de la tabla de empleados

In [3]:
def crear_tabla(conn):
    try:
        sql = ''' CREATE TABLE IF NOT EXISTS empleados (
                                        id integer PRIMARY KEY,
                                        nombre text NOT NULL,
                                        cargo text NOT NULL,
                                        salario real
                                    ); '''
        conn.cursor().execute(sql)
    except Error as e:
        print(e)

## Insercion de empleados

In [4]:
def insertar_empleado(conn, empleado):
    sql = ''' INSERT INTO empleados(nombre,cargo,salario)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, empleado)
    conn.commit()
    return cur.lastrowid

# Consulta de empleados

In [5]:
def consultar_empleados(conn):
    cur = conn.cursor()
    cur.execute("SELECT * FROM empleados")
    rows = cur.fetchall()
    for row in rows:
        print(row)

# Actualización de empleados

In [6]:
# Actualización de empleados
def actualizar_empleado(conn, empleado):
    sql = '''
    UPDATE empleados
    SET nombre = ?,
        cargo = ?,
        salario = ?
    WHERE id = ?
    '''
    cur = conn.cursor()
    try:
        cur.execute(sql, empleado)
        conn.commit()
        return "Empleado actualizado correctamente."
    except Exception as e:
        print(f"Error al actualizar empleado: {e}")
        return f"Error al actualizar empleado: {e}"


# Eliminación de un empleado

In [7]:
# Eliminación de un empleado
def eliminar_empleado(conn, id):
    sql = 'DELETE FROM empleados WHERE id = ?'
    cur = conn.cursor()
    try:
        cur.execute(sql, (id,))
        conn.commit()
        return "Empleado eliminado correctamente."
    except Exception as e:
        print(f"Error al eliminar empleado: {e}")
        return f"Error al eliminar empleado: {e}"

# Creación de la ventana de la aplicación

In [8]:
def crear_ventana():
    ventana = Tk()
    ventana.title("Gestión de empleados")
    ventana.geometry("800x600")
    return ventana

# Creación de botones y campos de texto para la interfaz de usuario

In [9]:
from tkinter import *
from tkinter import messagebox

def crear_interfaz(ventana, conn):
    # Configurar pesos para las filas y las columnas
    for i in range(10):
        ventana.grid_rowconfigure(i, weight=1)
    for i in range(4):
        ventana.grid_columnconfigure(i, weight=1)

    # Crear campos de texto para introducir información de empleados
    Label(ventana, text="Nombre del empleado:").grid(row=0, column=0, sticky=W)
    nombre_texto = StringVar()
    Entry(ventana, textvariable=nombre_texto).grid(row=1, column=0, sticky=E+W)

    Label(ventana, text="Cargo del empleado:").grid(row=0, column=1, sticky=W)
    cargo_texto = StringVar()
    Entry(ventana, textvariable=cargo_texto).grid(row=1, column=1, sticky=E+W)

    Label(ventana, text="Salario del empleado:").grid(row=0, column=2, sticky=W)
    salario_texto = StringVar()
    Entry(ventana, textvariable=salario_texto).grid(row=1, column=2, sticky=E+W)

    # Botón para insertar empleados
    Button(ventana, text="Insertar empleado", 
        command=lambda: insertar_empleado(conn, (nombre_texto.get(), cargo_texto.get(), float(salario_texto.get())))).grid(row=1, column=3, sticky=E+W)

    # Botón para consultar empleados
    def mostrar_empleados():
        # Consultar empleados y mostrar los resultados
        cur = conn.cursor()
        cur.execute("SELECT * FROM empleados")
        rows = cur.fetchall()
        resultados_texto.delete(1.0, END)
        for row in rows:
            resultados_texto.insert(END, str(row) + '\n')

    Button(ventana, text="Consultar empleados", command=mostrar_empleados).grid(row=2, column=3, sticky=E+W)

    # Crear un widget Text para mostrar los resultados de la consulta
    resultados_texto = Text(ventana)
    resultados_texto.grid(row=3, column=0, columnspan=4, rowspan=4, sticky=N+E+S+W)

    Label(ventana, text="ID del empleado a actualizar/eliminar:").grid(row=7, column=0, sticky=W)
    id_texto = StringVar()
    Entry(ventana, textvariable=id_texto).grid(row=8, column=0, sticky=E+W)

    # Botón para cargar empleados
    def cargar_empleado():
        cur = conn.cursor()
        cur.execute("SELECT * FROM empleados WHERE id=?", (int(id_texto.get()),))
        row = cur.fetchone()
        if row is not None:
            nombre_texto.set(row[1])
            cargo_texto.set(row[2])
            salario_texto.set(str(row[3]))
        else:
            messagebox.showerror("Error", "No se encontró un empleado con ese ID")

    Button(ventana, text="Cargar empleado", command=cargar_empleado).grid(row=8, column=1, sticky=E+W)

    # Botón para actualizar empleados
    def actualizar():
        respuesta = messagebox.askokcancel("Confirmar actualización", "¿Estás seguro de que quieres actualizar este empleado?")
        if respuesta:
            actualizar_empleado(conn, (nombre_texto.get(), cargo_texto.get(), float(salario_texto.get()), int(id_texto.get())))
            messagebox.showinfo("Información", "Empleado actualizado.")

    Button(ventana, text="Actualizar empleado", command=actualizar).grid(row=8, column=2, sticky=E+W)

    # Botón para eliminar empleados
    def eliminar():
        respuesta = messagebox.askokcancel("Confirmar eliminación", "¿Estás seguro de que quieres eliminar este empleado?")
        if respuesta:
            eliminar_empleado(conn, int(id_texto.get()))
            messagebox.showinfo("Información", "Empleado eliminado.")

    Button(ventana, text="Eliminar empleado", command=eliminar).grid(row=8, column=3, sticky=E+W)


if __name__ == '__main__':
    conn = crear_conexion()
    crear_tabla(conn)
    ventana = crear_ventana() 
    crear_interfaz(ventana, conn)
    ventana.mainloop()


2.6.0


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Fernanda\AppData\Local\Temp\ipykernel_9152\3063189966.py", line 73, in actualizar
    mensaje_actualizacion = actualizar_empleado(conn, (nombre_texto.get(), cargo_texto.get(), float(salario_texto.get()), int(id_texto.get())))
                                                                                              ^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: '90000/show'
